# Converting scikit-learn models to ONNX for use in Android apps

* ✍️ [Read the blog](https://towardsdatascience.com/deploying-scikit-learn-models-in-android-apps-with-onnx-b3adabe16bab)
* 🌐 [Checkout the GitHub repo](https://github.com/shubham0204/Scikit_Learn_Android_Demo)
* 📁 [Download this notebook](https://github.com/shubham0204/Google_Colab_Notebooks/blob/main/Sklearn_to_ONNX_Conversion.ipynb)

### 1️⃣ Installing `onnxruntime` and `skl2onnx`

In [ ]:
!pip install onnxruntime skl2onnx

### 2️⃣ Building the scikit-learn model

In [ ]:
import pandas as pd
import numpy as np

data = pd.read_csv( 'score.csv' )
data.head()

In [ ]:
from sklearn.linear_model import LinearRegression

X , y = data.values[ : , 0 ] , data.values[ : , 1 ]
X = np.expand_dims( X , axis=1 )

regressor = LinearRegression()
regressor.fit( X , y )

In [ ]:
print( f'Prediction for 8.5 hours is {regressor.predict([[8.5]])[0]}' )

### 3️⃣ Conversion to `.onnx`

In [ ]:
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType

# Specify an initial type for the model ( similar to input shape for the model )
initial_type = [ 
    ( 'input_study_hours' , FloatTensorType( [None,1] ) ) 
]

# Write the ONNX model to disk
converted_model = convert_sklearn( regressor , initial_types=initial_type )
with open( "sklearn_model.onnx", "wb" ) as f:
    f.write( converted_model.SerializeToString() )

### 4️⃣ Conversion of `.onnx` to `.ort` ( Optional )

As per [Scott McKay's comment](https://github.com/shubham0204/Scikit_Learn_Android_Demo/issues/1),

> It’s not necessary to convert to .ort format when using the ‘full’ ONNX Runtime package onnxruntime-android - you can use the onnx model.
> The conversion to .ort format is only necessary if using the smaller ‘mobile’ package onnxruntime-mobile, which has limited operators/types (based on popular dnn models used in mobile scenarios) to provide a smaller binary size. That package however does not include traditional ML operators that SciKit-Learn tends to use, so most likely it wouldn't be able to run a model that was converted from SKL.


In [ ]:
!python -m onnxruntime.tools.convert_onnx_models_to_ort sklearn_model.onnx